In [7]:
from yacht import Battle
import numpy as np
from keras import layers, Input, optimizers, losses
from keras.models import Model
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf
gpu_available = tf.test.is_gpu_available()

In [8]:
def one_hot(x):
    return np.where(x>0,1,0)


In [ ]:
class YachtGAN(Model):
    def __init__(self):
        pass
    
    def call(self, x):
        pass

In [9]:
def hot2fix(hot, cur_dice):
    '''
    hot(5,) is np.array((1,1,0,0,1)) --> len 5, tell which dice should fix
    cur_dice(6,) is apparently current dice.
    '''
    fix = np.zeros(6)
    j=1#j is value of dice
    for i in range(5):#i is count of dice
        while sum(cur_dice[:j])<i+1:
            j+=1
        if hot[i]:
            fix[j-1]+=1
    return fix


In [10]:
dice_input = Input(shape=(6,), name = 'dice')
dice_remain_input = Input(shape=(1,), name = 'dice_remain')
score_selected_input = Input(shape=(12,), name = 'score_selected')

# score_board_input = Input(shape=(12,), dtype = 'int32', name = 'score_board')
x = layers.concatenate([dice_input, dice_remain_input])
x = layers.Dense(64, activation='relu')(x)
x = layers.concatenate([x, score_selected_input])
dice_fix_output = layers.Dense(5,activation='one_hot', dynamic = True, name = 'dice_fix')(x)#When you use custom activation, use dynamic
x= layers.Dense(64, activation = 'relu')(x)
choose_output = layers.Dense(12, activation = 'softmax', name = 'choose')(x)
model = Model([dice_input, dice_remain_input, score_selected_input], [dice_fix_output, choose_output])



optimizer = optimizers.Adam(learning_rate=0.01)
huber_loss = losses.Huber()
action_probs_history = []
critic_value_history = []
rewards_history = []
running_reward = 0
episode_count = 0
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dice (InputLayer)               [(None, 6)]          0                                            
__________________________________________________________________________________________________
dice_remain (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 7)            0           dice[0][0]                       
                                                                 dice_remain[0][0]                
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 64)           512         concatenate_2[0][0]   

In [11]:

def cow_expect(cow_model,n=20):
    loss=0
    for i in range(n):
        cow=battle()
        dice_fix, choose = cow_model([
            tf.expand_dims(cow.dice,0),
            tf.expand_dims(cow.dice_remain,0),
            tf.expand_dims(cow.score_selected,0)])
        for turn in range(12):
            for roll in range(3):
                dice_fix, choose = cow_model([
                    tf.expand_dims(cow.dice,0),
                    tf.expand_dims(cow.dice_remain,0),
                    tf.expand_dims(cow.score_selected,0)])
                choose = np.array(choose)
                while cow.score_selected[choose[0].argmax()]:
                    choose[0][choose[0].argmax()]=0
                    loss+=1
                if sum(dice_fix[0])==5 or roll==2: break
                fix = hot2fix(dice_fix[0], cow.dice)
                cow.roll(fix=fix.astype(int))
            cow.turn(choose[0].argmax())
        loss+=cow.total_score()
    return loss
cow_expect(model)


437.7

In [31]:
n=20
for i in range(1000):  # Run until solved
    episode_reward = 0
    with tf.GradientTape() as tape:
        loss=tf.constant(0.)
        for i in range(n):
            cow=battle()
            dice_fix, choose = model([
                tf.expand_dims(cow.dice,0),
                tf.expand_dims(cow.dice_remain,0),
                tf.expand_dims(cow.score_selected,0)])
            for turn in range(12):
                for roll in range(3):
                    dice_fix, choose = model([
                        tf.expand_dims(cow.dice,0),
                        tf.expand_dims(cow.dice_remain,0),
                        tf.expand_dims(cow.score_selected,0)])
                    choose = np.array(choose)
                    while cow.score_selected[choose[0].argmax()]:
                        choose[0][choose[0].argmax()]=0
                        loss+=1
                    if sum(dice_fix[0])==5 or roll==2: break
                    fix = hot2fix(dice_fix[0], cow.dice)
                    cow.roll(fix=fix.astype(int))
                loss+=350 - cow.total_score()
                cow.turn(choose[0].argmax())
    print(loss)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

tf.Tensor(81209.0, shape=(), dtype=float32)


ValueError: No gradients provided for any variable: ['dense_2/kernel:0', 'dense_2/bias:0', 'dense_3/kernel:0', 'dense_3/bias:0', 'dice_fix/kernel:0', 'dice_fix/bias:0', 'choose/kernel:0', 'choose/bias:0'].

In [ ]:
class cow_Agent():
    def __init__(self, state_size, action_size):
        self.weight_backup      = "cartpole_weight.h5"
        self.state_size         = state_size
        self.action_size        = action_size
        self.memory             = deque(maxlen=2000)
        self.learning_rate      = 0.001
        self.gamma              = 0.95
        self.exploration_rate   = 1.0
        self.exploration_min    = 0.01
        self.exploration_decay  = 0.995
        self.brain              = self._build_model()

    def cow_expect(cow_model,n=20):

    total_loss = 0
    for i in range(n):
        cow=battle()
        loss=0
        dice_fix, choose = cow_model([
            tf.expand_dims(cow.dice,0),
            tf.expand_dims([cow.dice_remain],0),
            tf.expand_dims(cow.score_selected,0)])
        for turn in range(12):
            for roll in range(3):
                dice_fix, choose = cow_model([
                    tf.expand_dims(cow.dice,0),
                    tf.expand_dims(cow.dice_remain,0),
                    tf.expand_dims(cow.score_selected,0)])
                choose = np.array(choose)
                while cow.score_selected[choose[0].argmax()]:
                    choose[0][choose[0].argmax()]=0
                    loss+=1
                if sum(dice_fix[0])==5 or roll==2: break
                fix = hot2fix(dice_fix[0], cow.dice)
                cow.roll(fix=fix.astype(int))
            cow.turn(choose[0].argmax())
        total_loss+=cow.total_score()
    loss += 350 - total_loss/n
    return loss

In [ ]:

while True:  # Run until solved
    episode_reward = 0
    cow = battle()
    with tf.GradientTape() as tape:
        loss=0
        for timestep in range(1, max_steps_per_episode):
            loss_value = cow_expect(model)
                

            action_probs, critic_value = model(state)
            critic_value_history.append(critic_value[0, 0])

            # Sample action from action probability distribution
            action = np.random.choice(num_actions, p=np.squeeze(action_probs))
            action_probs_history.append(tf.math.log(action_probs[0, action]))

            rewards_history.append(reward)
            episode_reward += reward

            if done:
                break

        # Update running reward to check condition for solving
        running_reward = 0.05 * episode_reward + (1 - 0.05) * running_reward

        # Calculate expected value from rewards
        # - At each timestep what was the total reward received after that timestep
        # - Rewards in the past are discounted by multiplying them with gamma
        # - These are the labels for our critic
        returns = []
        discounted_sum = 0
        for r in rewards_history[::-1]:
            discounted_sum = r + gamma * discounted_sum
            returns.insert(0, discounted_sum)

        # Normalize
        returns = np.array(returns)
        returns = (returns - np.mean(returns)) / (np.std(returns) + eps)
        returns = returns.tolist()

        # Calculating loss values to update our network
        history = zip(action_probs_history, critic_value_history, returns)
        actor_losses = []
        critic_losses = []
        for log_prob, value, ret in history:
            # At this point in history, the critic estimated that we would get a
            # total reward = `value` in the future. We took an action with log probability
            # of `log_prob` and ended up recieving a total reward = `ret`.
            # The actor must be updated so that it predicts an action that leads to
            # high rewards (compared to critic's estimate) with high probability.
            diff = ret - value
            actor_losses.append(-log_prob * diff)  # actor loss

            # The critic must be updated so that it predicts a better estimate of
            # the future rewards.
            critic_losses.append(
                huber_loss(tf.expand_dims(value, 0), tf.expand_dims(ret, 0))
            )

        # Backpropagation
        loss_value = sum(actor_losses) + sum(critic_losses)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Clear the loss and reward history
        action_probs_history.clear()
        critic_value_history.clear()
        rewards_history.clear()

    # Log details
    episode_count += 1
    if episode_count % 10 == 0:
        template = "running reward: {:.2f} at episode {}"
        print(template.format(running_reward, episode_count))

    if running_reward > 195:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break